In [1]:
import os
import dotenv

project_dir = os.path.join(os.path.dirname(__vsc_ipynb_file__), os.pardir)
dotenv_path = os.path.join(project_dir, '.env')
dotenv.load_dotenv(dotenv_path)

# For rerun to work in a notebook the environment variable RERUN_NOTEBOOK_ASSET must be set to "inline" before importing the rerun package
print(os.getenv("RERUN_NOTEBOOK_ASSET"))

inline


In [2]:
import numpy as np
import rerun as rr
from numpy.random import default_rng

# Example of visualizing a 3D point set

In [10]:
rr.init("rerun_example_notebook")

rng = default_rng(12345)

positions = rng.uniform(-5, 5, size=[10, 3])
colors = rng.uniform(0, 255, size=[10, 3])
radii = rng.uniform(0, 1, size=[10])

rr.log("random", rr.Points3D(positions, colors=colors, radii=radii))

rr.notebook_show()

Viewer()

# Example of visualizing a 3D Tensor

In [11]:
rr.init("visualize_data_notebook")

rng = default_rng(12345)

rr.log(
    entity_path = "scan",
    entity = rr.Tensor(
        data = rng.uniform(-1, 1, size=[15, 10, 20]),
        dim_names = ["height", "width", "depth"])
)

rr.notebook_show()

Viewer()

# Example of visualizing a MRI scan from a DICOM file

In [4]:
from pydicom import read_file, FileDataset
from glob import glob

ROOT_SCAN_PATH = "/media/vlad/sda2/Projects/Me/Kaggle/RSNA-2024-Lumbar-Spine-Degenerative-Classification/data/raw/train_images/4003253"

In [5]:
# Read from dicom file

scan_views: dict[str, np.ndarray | dict[str, FileDataset]] = {}
for view_id in os.listdir(ROOT_SCAN_PATH):
    view_slices: dict[str, FileDataset] = {}
    view_tensor: list[np.ndarray] = []
    print(f"Reading from {ROOT_SCAN_PATH}/{view_id}:", end=" ")
    for slice_path in sorted(glob(os.path.join(ROOT_SCAN_PATH, view_id, "*.dcm")), key=lambda x: int(x.split("/")[-1].split(".")[0])):
        slice_id = slice_path.split("/")[-1]
        slice = read_file(slice_path)
        print(f"{slice_id}", end=", ")
        view_slices[slice_id] = slice
        view_tensor.append(slice.pixel_array)
    print()
    scan_views[view_id+"_dicom"] = view_slices
    scan_views[view_id+"_tensor"] = np.array(view_tensor).transpose(-1, 1, 0)
print(f"Read {sum([len(scan_views[view_id+"_dicom"].keys()) for view_id in os.listdir(ROOT_SCAN_PATH)])} slices from {ROOT_SCAN_PATH}")

Reading from /media/vlad/sda2/Projects/Me/Kaggle/RSNA-2024-Lumbar-Spine-Degenerative-Classification/data/raw/train_images/4003253/1054713880: 1.dcm, 2.dcm, 3.dcm, 4.dcm, 5.dcm, 6.dcm, 7.dcm, 8.dcm, 9.dcm, 10.dcm, 11.dcm, 12.dcm, 13.dcm, 14.dcm, 15.dcm, 
Reading from /media/vlad/sda2/Projects/Me/Kaggle/RSNA-2024-Lumbar-Spine-Degenerative-Classification/data/raw/train_images/4003253/2448190387: 1.dcm, 2.dcm, 3.dcm, 4.dcm, 5.dcm, 6.dcm, 7.dcm, 8.dcm, 9.dcm, 10.dcm, 11.dcm, 12.dcm, 13.dcm, 14.dcm, 15.dcm, 16.dcm, 17.dcm, 18.dcm, 19.dcm, 20.dcm, 21.dcm, 22.dcm, 23.dcm, 24.dcm, 25.dcm, 26.dcm, 27.dcm, 28.dcm, 29.dcm, 30.dcm, 31.dcm, 32.dcm, 33.dcm, 34.dcm, 35.dcm, 36.dcm, 37.dcm, 38.dcm, 39.dcm, 40.dcm, 41.dcm, 42.dcm, 43.dcm, 
Reading from /media/vlad/sda2/Projects/Me/Kaggle/RSNA-2024-Lumbar-Spine-Degenerative-Classification/data/raw/train_images/4003253/702807833: 1.dcm, 2.dcm, 3.dcm, 4.dcm, 5.dcm, 6.dcm, 7.dcm, 8.dcm, 9.dcm, 10.dcm, 11.dcm, 12.dcm, 13.dcm, 14.dcm, 15.dcm, 
Read 73 slices 

In [9]:
rr.init("visualize_mri_volume_notebook")

for view_id in os.listdir(ROOT_SCAN_PATH):
    rr.log(
        entity_path=f"view_{view_id}",
        entity=rr.Tensor(
            data=np.rot90(scan_views[f"{view_id}_tensor"], k=3),
            dim_names=["height", "width", "depth"]
            )
    )

for view_id in os.listdir(ROOT_SCAN_PATH):
    rr.blueprint.Horizontal(
        rr.blueprint.Spatial2DView(origin=f"/{view_id}")
    )


rr.notebook_show()

Viewer()